In [125]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
from tqdm.notebook import tqdm

In [4]:
%cd Stock

/home/dsc/Repos/TFM/Stock


In [3]:
%ls

'PW Tracker 15 Mar.xlsx'


In [59]:
stock = pd.read_excel('PW Tracker 15 Mar.xlsx')

In [5]:
stock.head()

,Country,PartNumber,Description / Family,Brand,Stock,Released Orders,Clean to Ship,Order Shorts,Open to Sell,Comments,Replenishment ETA,Current Status,Estimated Delivery
0,AT/BE/CH/DE/DK/ES/FI/FR/GB/IE/IT/NL/NO/PT/SE,66B8KAC6EU,D27-30(D20270FD0)27inch FHD Monitor-HDMI,Visual,0,0,0,0,0,Replenishment Planned,Unshipped 2021/5/21,5D11A99760 Monitor@2021/03/20;5D11A99760;DISP...,Unshipped 2021/5/21
1,AT/BE/DE/DK/ES/FI/FR/GB/IE/IT/NL/NO/PT/SE,66AEKAC1EU,D24-20(A20238FD0)23.8inch Monitor-HDMI,Visual,0,0,0,0,0,Replenishment Planned,TBD,SHORT : 5D10Z90828 Monitor@9999/12/31;5D10Z90...,TBD
2,BE/DE/AT/CH/DK/ES/FI/FR/ IT/NL/NO/PT/SE,ZA4Y0059SE,Lenovo TB-X605FC TAB 3G+32GWH-SE,Tablet,0,0,0,0,0,NaN,NaN,NaN,NaN
3,BE/DE/AT/CH/DK/ES/FI/FR/ IT/NL/NO/PT/SE,ZA4Y0072SE,Lenovo TB-X605FC TAB 3G+32GBL-SE,Tablet,0,0,0,0,0,Replenishment Planned,2021-03-28 00:00:00,300 Shipped,2021-03-28 00:00:00
4,BE/DE/AT/CH/DK/ES/FI/FR/ IT/NL/NO/PT/SE,ZA5H0064SE,Lenovo TB-8505X TAB 2G+32GGR-SE,Tablet,0,0,0,0,0,Replenishment Planned,2021-03-27 00:00:00,300 Shipped,2021-03-27 00:00:00


In [6]:
stock['Brand'].unique()

array(['Visual', 'Tablet', 'Option', 'Phone', 'AIO', 'Desktop',
       'Notebook', 'Workstation_DT'], dtype=object)

In [60]:
stock = stock[((stock['Country'] == 'Spain') | (stock['Country'] == 'Portugal')) &
    ((stock['Brand'] == 'Notebook') | (stock['Brand'] == 'Desktop'))]

In [8]:
len(stock)

190

In [9]:
stock.iloc[:,:5]

,Country,PartNumber,Description / Family,Brand,Stock
1620,Portugal,20RR0045PG,NB LN ThinkBook 13sIML I5 8G 256G 10H,Notebook,1
1621,Portugal,20S6000NPG,NoteBook TP T15 G1 I5 8G 10P,Notebook,3
1622,Portugal,20S6001XPG,NoteBook TP T15 G1 I7 16G 10P,Notebook,0
1623,Portugal,20TA002BPG,NB TP E14 Gen 2 I5 16G 512G 10P,Notebook,0
1624,Portugal,20TA002KPG,NB TP E14 Gen 2 I7 16G 512G 10P,Notebook,0
...,...,...,...,...,...
1818,Spain,82L50012SP,NB IP 5 Pro 16ACH6 R7 16G 1T 10H,Notebook,0
1819,Spain,82LM005VSP,NB IP 5 14ALC05 R5 16G 512G NOS,Notebook,0
1820,Spain,82LM005WSP,NB IP 5 14ALC05 R5 16G 512G 10H,Notebook,0
1821,Spain,82MS0027SP,NB YG Slim 7 Pro 14ACH5 R7 16G 1T 10H,Notebook,0


In [10]:
stock.iloc[:,8:9]

,Open to Sell
1620,1
1621,3
1622,0
1623,0
1624,0
...,...
1818,0
1819,0
1820,0
1821,0


In [11]:
pd.concat([stock.iloc[:,:5], stock.iloc[:,8:9]], axis=1)

,Country,PartNumber,Description / Family,Brand,Stock,Open to Sell
1620,Portugal,20RR0045PG,NB LN ThinkBook 13sIML I5 8G 256G 10H,Notebook,1,1
1621,Portugal,20S6000NPG,NoteBook TP T15 G1 I5 8G 10P,Notebook,3,3
1622,Portugal,20S6001XPG,NoteBook TP T15 G1 I7 16G 10P,Notebook,0,0
1623,Portugal,20TA002BPG,NB TP E14 Gen 2 I5 16G 512G 10P,Notebook,0,0
1624,Portugal,20TA002KPG,NB TP E14 Gen 2 I7 16G 512G 10P,Notebook,0,0
...,...,...,...,...,...,...
1818,Spain,82L50012SP,NB IP 5 Pro 16ACH6 R7 16G 1T 10H,Notebook,0,0
1819,Spain,82LM005VSP,NB IP 5 14ALC05 R5 16G 512G NOS,Notebook,0,0
1820,Spain,82LM005WSP,NB IP 5 14ALC05 R5 16G 512G 10H,Notebook,0,0
1821,Spain,82MS0027SP,NB YG Slim 7 Pro 14ACH5 R7 16G 1T 10H,Notebook,0,0


In [61]:
stock = stock.iloc[:,[0,1,2,3,4,8]].copy()

In [27]:
stock['Date'] = np.array('2021-03-13', dtype=np.datetime64)

In [45]:
del stock['Date']

In [54]:
pd.to_datetime('2021-02-08').isocalendar()[1]

6

In [62]:
stock['Year'] = 2021
stock['Month'] = 3
stock['Day'] = 13
stock['Week'] = pd.to_datetime('2021-02-08').isocalendar()[1]

In [63]:
stock

,Country,PartNumber,Description / Family,Brand,Stock,Open to Sell,Year,Month,Day,Week
1620,Portugal,20RR0045PG,NB LN ThinkBook 13sIML I5 8G 256G 10H,Notebook,1,1,2021,3,13,6
1621,Portugal,20S6000NPG,NoteBook TP T15 G1 I5 8G 10P,Notebook,3,3,2021,3,13,6
1622,Portugal,20S6001XPG,NoteBook TP T15 G1 I7 16G 10P,Notebook,0,0,2021,3,13,6
1623,Portugal,20TA002BPG,NB TP E14 Gen 2 I5 16G 512G 10P,Notebook,0,0,2021,3,13,6
1624,Portugal,20TA002KPG,NB TP E14 Gen 2 I7 16G 512G 10P,Notebook,0,0,2021,3,13,6
...,...,...,...,...,...,...,...,...,...,...
1818,Spain,82L50012SP,NB IP 5 Pro 16ACH6 R7 16G 1T 10H,Notebook,0,0,2021,3,13,6
1819,Spain,82LM005VSP,NB IP 5 14ALC05 R5 16G 512G NOS,Notebook,0,0,2021,3,13,6
1820,Spain,82LM005WSP,NB IP 5 14ALC05 R5 16G 512G 10H,Notebook,0,0,2021,3,13,6
1821,Spain,82MS0027SP,NB YG Slim 7 Pro 14ACH5 R7 16G 1T 10H,Notebook,0,0,2021,3,13,6


In [2]:
stock.to_excel('stock.xlsx', index=False)

NameError: name 'stock' is not defined

In [12]:
stock = pd.read_excel('PW Tracker _ 11th Mar 2015.xlsx', sheet_name='Daily Stock Report')

In [13]:
stock.columns = np.linspace(1,24,24).astype('int')

In [15]:
stock = stock[stock[2] == 'Spain'].iloc[:,[1,2,3,4,5,9]].copy()

In [16]:
columns = ['Country', 'PartNumber', 'Description / Family', 'Brand', 'Stock', 'Open to Sell']

In [18]:
stock.columns = columns

In [19]:
stock

,Country,PartNumber,Description / Family,Brand,Stock,Open to Sell
661,Spain,F0AN000SSP,A540,Desktop,2,1
662,Spain,F0AM001USP,A740,Desktop,2,2
663,Spain,F0AU00CPSP,B50-30,Desktop,0,0
664,Spain,57331501,C260,Desktop,0,0
665,Spain,F0B4007USP,C40-30,Desktop,0,0
666,Spain,90B8002SSP,H30-50,Desktop,0,0
667,Spain,90B6007USP,H50-50,Desktop,0,0
668,Spain,90B6007WSP,H50-50,Desktop,0,0
669,Spain,80FL000VSP,Flex 2 Pro-15,Notebook,5,5
670,Spain,59413057,U330Touch,Notebook,0,0


In [30]:
%cd TFM

/home/dsc/Repos/TFM


In [2]:
import os
path = '/home/dsc/Repos/TFM/Stock/'
files = os.listdir(path)
print (files)

['2015', '2016']


In [113]:
#Esta función devuelve una tupla con la ruta del fichero y luego nos quedamos con la 'fecha' del fichero
def file_iterator(path):
    """
    DESCRIPTION
      This function retreives all files in a specific folder.
    ARGUMENTS
      path: Where we want to get all files
    RETURN
      A list of tuples, first element is the path and the second is the file name without any extension
    """
    return [(os.path.join(path,sub,file),file.split('.')[0]) for sub in os.listdir(path) 
            for file in os.listdir(os.path.join(path,sub))]

In [114]:
def file_extractor(files):
    """
    DESCRIPTION
      This function return a concatenated DataFrame with all desired columns
    ARGUMENTS
      files: This is list of tuples, first element is the path and the second is the file name without any extension
    RETURN
      A concatenated DataFrame with 10 columns based on the files contained in the argument
    """
    
    #A list of DataFrames, where then we will concatenated
    stock = []
    #This is the returned DataFrame
    pd_stock = pd.DataFrame()
    
    #Progression bar
    pbar = tqdm()
    pbar.reset(len(files))
    
    #Files is a Tuple made up of (path,name without extension)
    for path, name in files:
        #Read the file
        aux_stock = pd.read_excel(path, sheet_name='Daily Stock Report')        
        #Get the desired columns filtered by Spain
        aux_stock = aux_stock[aux_stock.iloc[:,1] == 'Spain'].iloc[:,[1,2,3,4,5,9]]
        
        #We will use the file name for creating the Date, Year, Month, Day and Week Number
        year, month, day = name.split('-')
        week = pd.to_datetime(name).isocalendar()[1]
        #Add the Date columns
        aux_stock['Year'] = year
        aux_stock['Month'] = month
        aux_stock['Day'] = day
        aux_stock['Week'] = week
        
        #We add this DataFrame in a list for concatenating purposes
        stock.append(aux_stock)
        #Progress bar
        pbar.update()
    
    
    #Progress bar
    pbar.refresh()
    #Concatenated all DataFrames
    pd_stock = pd.concat(stock, ignore_index=True)
    #Rename the columns
    columns = ['Country', 'PartNumber', 'Description / Family', 
               'Brand', 'Stock', 'Open to Sell', 'Year', 'Month', 'Day', 'Week']
    pd_stock.columns = columns
    #Return the final DataFrame in the desired order
    return pd_stock[['Year', 'Month', 'Day', 'Week','Country', 'PartNumber', 'Description / Family', 
               'Brand', 'Stock', 'Open to Sell']]

In [147]:
def file_extractor(files):
    
    stock = []
    pd_stock = pd.DataFrame()
    
    #Barrar de progreso y la preparamos
    pbar = tqdm()
    pbar.reset(len(files))
    
    #Files es una tupla de ruta + nombre del fichero sin extensión
    for path, name in files:
        aux_stock = pd.read_excel(path, sheet_name='Daily Stock Report')        
        #Cojemos las columnas que nos interesan y filtrado por país
        aux_stock = aux_stock[aux_stock.iloc[:,1] == 'Spain'].iloc[:,[1,2,3,4,5,9]]
        #Ahora añadimos el año, mes, día y día de la semana
        year, month, day = name.split('-')
        #El día de la semana está en la segunda posición
        week = pd.to_datetime(name).isocalendar()[1]
        aux_stock['Year'] = year
        aux_stock['Month'] = month
        aux_stock['Day'] = day
        aux_stock['Week'] = week
        
        
        
        #Es mejor meter todos los DataFrames en una lista y luego concatenarlos del tirón
        stock.append(aux_stock)
        
        
        
        #Barra de progreso
        pbar.update()
    
    
    #Barra de progreso
    pbar.refresh()
    
    pd_stock = pd.concat(stock, ignore_index=True)
    
    #ultima parte renombrar las columnas
    columns = ['Country', 'PartNumber', 'Description / Family', 
               'Brand', 'Stock', 'Open to Sell', 'Year', 'Month', 'Day', 'Week']
    
    pd_stock.columns = columns
    
    
    
    return pd_stock[['Year', 'Month', 'Day', 'Week','Country', 'PartNumber', 'Description / Family', 
               'Brand', 'Stock', 'Open to Sell']]

In [ ]:
lista = file_iterator(path)

In [148]:

stock = file_extractor(lista[0:2])

In [132]:
columns = ['Country', 'PartNumber', 'Description / Family', 'Brand', 'Stock', 'Open to Sell', 'Year', 'Month', 'Day', 'Week']
stock.columns = columns

In [149]:
stock.head(40)

,Year,Month,Day,Week,Country,PartNumber,Description / Family,Brand,Stock,Open to Sell
0,2015,10,19,43,Spain,F0AN000SSP,A540,Desktop,6,6
1,2015,10,19,43,Spain,F0AN004ASP,A540,Desktop,10,10
2,2015,10,19,43,Spain,F0AM001USP,A740,Desktop,0,0
3,2015,10,19,43,Spain,F0AM009BSP,A740,Desktop,10,10
4,2015,10,19,43,Spain,F0AU00CPSP,B50-30,Desktop,6,6
5,2015,10,19,43,Spain,F0B4007USP,C40-30,Desktop,1,0
6,2015,10,19,43,Spain,90C2004FSP,H30-00,Desktop,4,4
7,2015,10,19,43,Spain,90B8002SSP,H30-50,Desktop,4,4
8,2015,10,19,43,Spain,90B6007WSP,H50-50,Desktop,4,4
9,2015,10,19,43,Spain,90B600AESP,H50-50,Desktop,8,8
